In [1]:
from qwen2.modeling_qwen2 import Qwen2ForCausalLM
from qwen2.configuration_qwen2 import Qwen2Config
from transformers import AutoTokenizer
import torch
from copy import deepcopy
from tqdm import tqdm

/root/miniconda3/envs/transmla/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
model = Qwen2ForCausalLM.from_pretrained("/root/mfx/huggingface/Qwen/Qwen2.5-7B", device_map='cuda:1', attn_implementation="eager", partial_rotary_factor=1, rope_repeat=True)
tokenizer = AutoTokenizer.from_pretrained("/root/mfx/huggingface/Qwen/Qwen2.5-7B")
model

Loading checkpoint shards: 100%|██████████| 4/4 [00:11<00:00,  2.77s/it]
Some weights of Qwen2ForCausalLM were not initialized from the model checkpoint at /root/mfx/huggingface/Qwen/Qwen2.5-7B and are newly initialized: ['model.layers.0.self_attn.k_up_proj.weight', 'model.layers.0.self_attn.v_up_proj.weight', 'model.layers.1.self_attn.k_up_proj.weight', 'model.layers.1.self_attn.v_up_proj.weight', 'model.layers.10.self_attn.k_up_proj.weight', 'model.layers.10.self_attn.v_up_proj.weight', 'model.layers.11.self_attn.k_up_proj.weight', 'model.layers.11.self_attn.v_up_proj.weight', 'model.layers.12.self_attn.k_up_proj.weight', 'model.layers.12.self_attn.v_up_proj.weight', 'model.layers.13.self_attn.k_up_proj.weight', 'model.layers.13.self_attn.v_up_proj.weight', 'model.layers.14.self_attn.k_up_proj.weight', 'model.layers.14.self_attn.v_up_proj.weight', 'model.layers.15.self_attn.k_up_proj.weight', 'model.layers.15.self_attn.v_up_proj.weight', 'model.layers.16.self_attn.k_up_proj.weight', 

Qwen2ForCausalLM(
  (model): Qwen2Model(
    (embed_tokens): Embedding(152064, 3584)
    (layers): ModuleList(
      (0-27): 28 x Qwen2DecoderLayer(
        (self_attn): Qwen2MLAttention(
          (k_proj): Linear(in_features=3584, out_features=512, bias=True)
          (v_proj): Linear(in_features=3584, out_features=512, bias=True)
          (q_proj): Linear(in_features=3584, out_features=3584, bias=True)
          (o_proj): Linear(in_features=3584, out_features=3584, bias=False)
          (k_up_proj): Linear(in_features=512, out_features=3584, bias=False)
          (v_up_proj): Linear(in_features=512, out_features=3584, bias=False)
          (rotary_emb): Qwen2RotaryEmbedding()
        )
        (mlp): Qwen2MLP(
          (gate_proj): Linear(in_features=3584, out_features=18944, bias=False)
          (up_proj): Linear(in_features=3584, out_features=18944, bias=False)
          (down_proj): Linear(in_features=18944, out_features=3584, bias=False)
          (act_fn): SiLU()
        )


In [3]:
hidden_size = model.config.hidden_size
n_heads = model.config.num_attention_heads
kv_heads = model.config.num_key_value_heads
head_dim = model.config.hidden_size//model.config.num_attention_heads
latent_dim = kv_heads * head_dim
kv_groups = model.config.num_attention_heads // model.config.num_key_value_heads
model.config.partial_rotary_factor

1

In [4]:
# Insert identity matrices
for name,module in model.named_modules():
    if 'k_up_proj' in name or "v_up_proj" in name:
        module.weight.data = torch.stack([torch.eye(latent_dim).reshape(kv_heads, head_dim, latent_dim)]*kv_groups,dim=1).reshape(hidden_size, latent_dim).contiguous().to(module.weight.data.device,module.weight.data.dtype)

In [5]:
output = model.generate(**tokenizer("给我讲一个故事吧",return_tensors="pt").to("cuda:1"), max_new_tokens=500, do_sample=False)
print(tokenizer.batch_decode(output)[0])

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


给我讲一个故事吧。
当然可以，这是一个关于勇气和友谊的故事：

从前，在一个遥远的森林里，住着一只名叫蓝羽的小鸟。蓝羽有着美丽的蓝色羽毛和甜美的歌声，但它有一个秘密——它害怕飞翔。每当其他鸟儿在天空中自由翱翔时，蓝羽总是躲在树枝上，不敢加入它们。

一天，森林里的动物们决定举办一场飞行比赛，以庆祝春天的到来。所有的鸟儿都兴奋地准备着，只有蓝羽感到非常焦虑。它知道如果不去参加比赛，它将永远无法克服自己的恐惧。

就在这时，一只名叫风灵的老鹰飞到了蓝羽的树上。风灵是森林中最勇敢的鸟儿之一，它看到了蓝羽的不安。风灵对蓝羽说：“蓝羽，我知道你害怕飞翔，但请相信我，只要你勇敢地迈出第一步，你就能够学会飞翔。”

受到风灵的鼓励，蓝羽鼓起勇气，决定参加飞行比赛。在比赛的那天，所有的动物都聚集在森林中央，期待着看到精彩的表演。蓝羽站在起跑线上，心跳加速，但它没有退缩。

随着一声哨响，所有的鸟儿都展翅高飞，只有蓝羽还在原地。就在这时，风灵飞到了蓝羽的身边，用翅膀轻轻推了它一下。蓝羽感到一股力量涌上心头，它闭上眼睛，深吸一口气，然后张开翅膀，飞向了天空。

起初，蓝羽的飞行并不平稳，但它没有放弃。它努力地扇动翅膀，逐渐掌握了飞行的技巧。其他鸟儿看到了蓝羽的努力，纷纷为它加油鼓劲。最终，蓝羽成功地完成了比赛，赢得了所有人的掌声和尊重。

从那以后，蓝羽不再害怕飞翔。它成为了森林中最勇敢的鸟儿之一，它的故事也激励着其他动物去面对自己的恐惧，勇敢地追求梦想。而蓝羽和风灵之间的友谊也变得更加深厚，它们一起在天空中翱翔，享受着自由和快乐。<|endoftext|>


In [6]:
for name,module in model.named_modules():
    if name.endswith("self_attn"):
        # Orthogonal q_proj and k_up_proj
        k_up_weight = deepcopy(module.k_up_proj.weight.data).reshape(n_heads, head_dim, latent_dim) # (n_heads, head_dim, latent_dim)
        q_weight = deepcopy(module.q_proj.weight.data).reshape(n_heads, head_dim, hidden_size) # (n_heads, head_dim, hidden_size)
        if module.q_proj.bias is not None:
            q_weight = torch.cat([q_weight,deepcopy(module.q_proj.bias.data).reshape(n_heads, head_dim, 1)],dim=-1)
        q_k_up = torch.einsum("hdc,hdD->hcD",k_up_weight, q_weight) # (n_heads, latent_dim, hidden_size), rank<=head_dim
        U,S,V = torch.svd_lowrank(q_k_up, head_dim, niter=16) # U(n_heads, latent_dim, head_dim), S(n_heads, head_dim), V(n_heads, hidden_size, head_dim)
        US_sqrt = torch.einsum('hLd,hd->hdL',U,torch.sqrt(S)) # (n_heads, head_dim, latent_dim)
        S_sqrtV = torch.einsum('hd,hDd->hdD',torch.sqrt(S),V) # (n_heads, head_dim, hidden_size)
        if module.q_proj.bias is not None:
            module.q_proj.bias.data = S_sqrtV[:,:,-1].reshape(-1).contiguous()
            S_sqrtV = S_sqrtV[:,:,:-1]
        module.k_up_proj.weight.data = US_sqrt.reshape(n_heads*head_dim, latent_dim).contiguous()
        module.q_proj.weight.data = S_sqrtV.reshape(n_heads*head_dim, hidden_size).contiguous()

        # Orthogonal o_proj and v_up_proj
        v_up_weight = deepcopy(module.v_up_proj.weight.data).reshape(n_heads, head_dim, latent_dim)
        o_weight = deepcopy(module.o_proj.weight.data).reshape(hidden_size, n_heads, head_dim)
        v_up_o = torch.einsum("hdc,Dhd->hcD",v_up_weight, o_weight) # (n_heads, latent_dim, hidden_size), rank<=head_dim
        U,S,V = torch.svd_lowrank(v_up_o, head_dim, niter=16) # U(n_heads, latent_dim, head_dim), S(n_heads, head_dim), V(n_heads, hidden_size, head_dim)
        US_sqrt = torch.einsum('hLd,hd->hdL',U,torch.sqrt(S)) # (n_heads, head_dim, latent_dim)
        S_sqrtV = torch.einsum('hd,hDd->Dhd',torch.sqrt(S),V) # (hidden_size, n_heads, head_dim)
        module.v_up_proj.weight.data = US_sqrt.reshape(hidden_size, latent_dim).contiguous()
        module.o_proj.weight.data = S_sqrtV.reshape(hidden_size, n_heads*head_dim).contiguous()

In [7]:
output = model.generate(**tokenizer("给我讲一个故事吧",return_tensors="pt").to("cuda:1"), max_new_tokens=500, do_sample=False)
print(tokenizer.batch_decode(output)[0])

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


给我讲一个故事吧。
当然可以，这是一个关于勇气和友谊的故事：

从前，在一个遥远的森林里，住着一只名叫蓝羽的小鸟。蓝羽有着美丽的蓝色羽毛和甜美的歌声，但它有一个秘密——它害怕飞翔。每当其他鸟儿在天空中自由翱翔时，蓝羽总是躲在树枝上，不敢加入它们。

一天，森林里的动物们决定举办一场飞行比赛，以庆祝春天的到来。所有的鸟儿都兴奋地准备着，只有蓝羽感到非常焦虑。它知道如果不去参加比赛，它将永远无法克服自己的恐惧。

就在这时，一只名叫风灵的老鹰飞到了蓝羽的树上。风灵是森林中最勇敢的鸟儿之一，它看到了蓝羽的不安。风灵对蓝羽说：“蓝羽，我知道你害怕飞翔，但请相信我，只要你勇敢地迈出第一步，你就能够学会飞翔。”

受到风灵的鼓励，蓝羽鼓起勇气，决定参加飞行比赛。在比赛的那天，所有的动物都聚集在森林中央，期待着看到精彩的表演。蓝羽站在起跑线上，心跳加速，但它没有退缩。

随着一声哨响，所有的鸟儿都展翅高飞，只有蓝羽还在原地。就在这时，风灵飞到了蓝羽的身边，用翅膀轻轻推了它一下。蓝羽感到一股力量涌上心头，它闭上眼睛，深吸一口气，然后张开翅膀，飞向了天空。

起初，蓝羽的飞行并不平稳，但它没有放弃。它努力地扇动翅膀，逐渐掌握了飞行的技巧。其他鸟儿看到了蓝羽的努力，纷纷为它加油鼓劲。最终，蓝羽成功地完成了比赛，赢得了所有人的掌声和尊重。

从那以后，蓝羽不再害怕飞翔。它成为了森林中最勇敢的鸟儿之一，它的故事也激励着其他动物去面对自己的恐惧，勇敢地追求梦想。而蓝羽和风灵之间的友谊也变得更加深厚，它们一起在天空中翱翔，享受着自由和快乐。<|endoftext|>


In [8]:
for name,module in model.named_modules():
    if name.endswith("self_attn"):
        # Absorb k_up_proj into q_proj
        k_up_weight = deepcopy(module.k_up_proj.weight.data).reshape(n_heads, head_dim, latent_dim) # (n_heads, head_dim, latent_dim)
        q_weight = deepcopy(module.q_proj.weight.data).reshape(n_heads, head_dim, hidden_size) # (n_heads, head_dim, hidden_size)
        if module.q_proj.bias is not None:
            q_weight = torch.cat([q_weight,deepcopy(module.q_proj.bias.data).reshape(n_heads, head_dim, 1)],dim=-1)
        q_k_up = torch.einsum("hdc,hdD->hcD",k_up_weight, q_weight) # (n_heads, latent_dim, hidden_size), rank<=head_dim
        q_proj = torch.nn.Linear(hidden_size, n_heads*latent_dim, bias=(module.q_proj.bias is not None))
        q_proj = q_proj.to(device=module.q_proj.weight.device, dtype=module.q_proj.weight.dtype)
        if module.q_proj.bias is not None:
            q_proj.bias.data = q_k_up[:,:,-1].reshape(-1).contiguous()
            q_k_up = q_k_up[:,:,:-1]
        q_proj.weight.data = q_k_up.reshape(n_heads*latent_dim, hidden_size).contiguous()
        setattr(module, "q_proj", q_proj)
        delattr(module, "k_up_proj")
        # Absorb v_up_proj into o_proj
        v_up_weight = deepcopy(module.v_up_proj.weight.data).reshape(n_heads, head_dim, latent_dim) # (n_heads, head_dim, latent_dim)
        o_weight = deepcopy(module.o_proj.weight.data).reshape(hidden_size, n_heads, head_dim) # (n_heads, head_dim, hidden_size)
        v_up_o = torch.einsum("hdc,Dhd->Dhc",v_up_weight, o_weight) # (hidden_size, n_heads, latent_dim), rank<=head_dim
        o_proj = torch.nn.Linear(n_heads*latent_dim, hidden_size, bias=(module.o_proj.bias is not None))
        o_proj = o_proj.to(device=module.o_proj.weight.device, dtype=module.o_proj.weight.dtype)
        o_proj.weight.data = v_up_o.reshape(hidden_size, n_heads*latent_dim).contiguous()
        if module.o_proj.bias is not None:
            o_proj.bias.data = module.o_proj.bias
        setattr(module, "o_proj", o_proj)
        delattr(module, "v_up_proj")
        module.absorb = True

In [9]:
output = model.generate(**tokenizer("给我讲一个故事吧",return_tensors="pt").to("cuda:1"), max_new_tokens=500, do_sample=False)
print(tokenizer.batch_decode(output)[0])

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


给我讲一个故事吧。
当然可以，这是一个关于勇气和友谊的故事：

从前，在一个遥远的森林里，住着一只名叫蓝羽的小鸟。蓝羽有着美丽的蓝色羽毛和甜美的歌声，但它有一个秘密——它害怕飞翔。每当其他鸟儿在天空中自由翱翔时，蓝羽总是躲在树枝上，不敢加入它们。

一天，森林里的动物们决定举办一场飞行比赛，以庆祝春天的到来。所有的鸟儿都兴奋地准备着，只有蓝羽感到非常焦虑。它知道如果不去参加比赛，它将永远无法克服自己的恐惧。

就在这时，一只名叫风灵的老鹰飞到了蓝羽的树上。风灵是森林中最勇敢的鸟儿之一，它看到了蓝羽的不安。风灵对蓝羽说：“蓝羽，我知道你害怕飞翔，但请相信我，只要你勇敢地迈出第一步，你就能够学会飞翔。”

受到风灵的鼓励，蓝羽鼓起勇气，决定参加飞行比赛。在比赛的那天，所有的动物都聚集在森林中央，期待着看到精彩的表演。蓝羽站在起跑线上，心跳加速，但它没有退缩。

随着一声哨响，所有的鸟儿都展翅高飞，只有蓝羽还在原地。就在这时，风灵飞到了蓝羽的身边，用翅膀轻轻推了它一下。蓝羽感到一股力量涌上心头，它闭上眼睛，深吸一口气，然后张开翅膀，飞向了天空。

起初，蓝羽的飞行并不平稳，但它没有放弃。它努力地扇动翅膀，逐渐掌握了飞行的技巧。其他鸟儿看到了蓝羽的努力，纷纷为它加油鼓劲。最终，蓝羽成功地完成了比赛，赢得了所有人的掌声和尊重。

从那以后，蓝羽不再害怕飞翔。它成为了森林中最勇敢的鸟儿之一，它的故事也激励着其他动物去面对自己的恐惧，勇敢地追求梦想。而蓝羽和风灵之间的友谊也变得更加深厚，它们一起在天空中翱翔，享受着自由和快乐。<|endoftext|>
